In [7]:
import sqlite3 as lite
import pandas as pd

con = lite.connect('./data/checking-logs.sqlite') # подключение БД к Питон

In [8]:
test_result = pd.io.sql.read_sql("""
    SELECT 'before' AS 'times', avg(diff) AS avg_diff
    FROM (SELECT uid, avg(delta) AS diff
        FROM (SELECT cast(24 * (julianday(first_commit_ts) - julianday(deadlines)) AS integer) AS delta, uid
            FROM (SELECT t.uid, labname, first_commit_ts, first_view_ts,
                datetime(deadlines, 'unixepoch') AS deadlines
                FROM test AS t
                LEFT JOIN deadlines AS d
                ON t.labname = d.labs)
                WHERE labname NOT IN ('project1') AND first_commit_ts < first_view_ts)
                GROUP BY uid)
    union
    SELECT 'after' AS 'times', avg(diff) AS avg_diff
    FROM (SELECT uid, avg(delta) AS diff
        FROM (SELECT cast(24 * (julianday(first_commit_ts) - julianday(deadlines)) AS integer) AS delta, uid
            FROM (SELECT t.uid, labname, first_commit_ts, first_view_ts,
                datetime(d.deadlines, 'unixepoch') AS deadlines
                FROM test AS t
                LEFT JOIN deadlines AS d
                ON t.labname = d.labs)
                WHERE labname NOT IN ('project1') AND first_commit_ts > first_view_ts
                AND uid IN (SELECT uid
                            FROM test
                            WHERE labname NOT IN ('project1') AND first_commit_ts < first_view_ts
                GROUP BY uid))
        GROUP BY uid)
    """, con)
test_result

,times,avg_diff
0,after,-99.523810
1,before,-66.047619


In [9]:
control_result = pd.io.sql.read_sql("""
    SELECT 'before' AS 'times', avg(diff) AS avg_diff
    FROM (SELECT uid, avg(delta) AS diff
        FROM (SELECT cast(24 * (julianday(first_commit_ts) - julianday(deadlines)) AS integer) AS delta, uid
            FROM (SELECT c.uid, labname, first_commit_ts, first_view_ts,
                datetime(deadlines, 'unixepoch') AS deadlines
                FROM control AS c
                LEFT JOIN deadlines AS d
                ON c.labname = d.labs)
                WHERE labname NOT IN ('project1') AND first_commit_ts < first_view_ts)
                GROUP BY uid)
    union
    SELECT 'after' AS 'times', avg(diff) AS avg_diff
    FROM (SELECT uid, avg(delta) AS diff
        FROM (SELECT cast(24 * (julianday(first_commit_ts) - julianday(deadlines)) AS integer) AS delta, uid
            FROM (SELECT c.uid, labname, first_commit_ts, first_view_ts,
                datetime(d.deadlines, 'unixepoch') AS deadlines
                FROM control AS c
                LEFT JOIN deadlines AS d
                ON c.labname = d.labs)
            WHERE labname NOT IN ('project1') AND first_commit_ts > first_view_ts
            AND uid IN (SELECT uid
                        FROM control
                        WHERE labname NOT IN ('project1') AND first_commit_ts < first_view_ts
                    GROUP BY uid))
        GROUP BY uid)
    """, con)
control_result

,times,avg_diff
0,after,-99.322222
1,before,-98.033333


In [10]:
con.close()